# Convolutional Neural Network with Batch Normalization

### Full Process Jupyter Notebook

### Step 0: Import Packages

In [1]:
from keras.models import Sequential
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.utils.np_utils import to_categorical
from keras import metrics
from keras.optimizers import SGD,RMSprop,Adam
from keras.callbacks import EarlyStopping
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
import matplotlib
matplotlib.use('agg') 
import matplotlib.pyplot as plt
import numpy
from keras.datasets import cifar10
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

Using TensorFlow backend.


### Step 1: Import data from Cifar10

In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

### Step 2: Data preprocessing

We will do data preprocessing on our cifar10 data. We will normalize our data by divided them by 225, which can make the image value change from 1-225 to 0-1. 

In [3]:
# standard normalize the inputs
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

### Step 3: One-hot encode outputs

In [4]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

### Step 4: Build the CNN model with batch normalization

We add several layers in our structure, which include convoluntional layers, max pooling layers, batch normalization layers. Batch Normalization means each layer takes a mini batch data set and stores the running average of those means and standard deviations. This layer helps in trainning a deeper neural network. We aso use max pooling, but we only add two to three pooling layers since it may result in losing information features of the image. Then we set epoch = 40 and iliterate about 200000 times in our model.

In [5]:
# Construct the model which include convolutional layer, max pooling layer and batch normalization layer.
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=(3, 32, 32), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(400))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


# Compile the model and set epochs = 40.
epochs = 40
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 32, 32)        1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 32, 32)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 16, 16)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 16, 16)        64        
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 16, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 16, 16)        36928     
__________

### Step5: Fit the model and calculate the test accuracy

We train the model using training data with size 40,000, and validate it using small batches (32) with a total of 10,000 due to GPU capacity. Test accuracy is calculated and below is the result.

In [6]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32)
# Accuracy of the BN-model
acc = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (acc[1]*100))
#This framework is ad

Train on 50000 samples, validate on 10000 samples
Epoch 1/40
50000/50000 [==============================] - 44s 876us/step - loss: 2.1526 - acc: 0.2917 - val_loss: 2.9448 - val_acc: 0.1888
Epoch 2/40
50000/50000 [==============================] - 42s 846us/step - loss: 1.6420 - acc: 0.4155 - val_loss: 1.5542 - val_acc: 0.4409
Epoch 3/40
50000/50000 [==============================] - 42s 839us/step - loss: 1.4399 - acc: 0.4857 - val_loss: 1.3172 - val_acc: 0.5205
Epoch 4/40
50000/50000 [==============================] - 42s 843us/step - loss: 1.3229 - acc: 0.5321 - val_loss: 1.8483 - val_acc: 0.4089
Epoch 5/40
50000/50000 [==============================] - 42s 843us/step - loss: 1.2452 - acc: 0.5587 - val_loss: 1.3718 - val_acc: 0.5154
Epoch 6/40
50000/50000 [==============================] - 41s 824us/step - loss: 1.1870 - acc: 0.5790 - val_loss: 1.3269 - val_acc: 0.5405
Epoch 7/40
50000/50000 [==============================] - 42s 839us/step - loss: 1.1415 - acc: 0.5981 - val_loss: 1.

### Conclusion

We find that the result of our BN model is not as good as other implementations. But the batch normalization method indeed can improve our model and simplfy the computation complexity. 